In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import openai
from openai import OpenAI
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/Speech-NER/train.pkl", 'rb') as file:
    train = pickle.load(file)

with open("/content/drive/MyDrive/Speech-NER/test.pkl", 'rb') as file:
    test = pickle.load(file)

with open("/content/drive/MyDrive/Speech-NER/aug.pkl", 'rb') as file:
    aug = pickle.load(file)

In [ ]:
seed_data = [[], []]
for i in range(len(train[0])):
  if train[1][i] != []:
    seed_data[0].append(train[0][i])
    seed_data[1].append(train[1][i])

In [ ]:
list_of_tags = []
for i in seed_data[1]:
  temp = []
  for j in i:
    if j['label'] not in list_of_tags:
      temp.append(j['label'])
  temp = list(set(sorted(temp)))
  list_of_tags.append(tuple(temp))

categorised_data = {}
for i in list_of_tags:
  categorised_data[i] = []

In [ ]:
def replace_tags_with_pad(sentences, tags_list):
    new_sentences = []

    for sentence, tags in zip(sentences, tags_list):
        sentence_chars = list(sentence)

        for tag in tags[::-1]:
            start = tag['start']
            end = tag['end']
            sentence_chars[start:end] = ""

        new_sentence = ''.join(sentence_chars)
        new_sentences.append(new_sentence)

    return new_sentences

struct_data = replace_tags_with_pad(seed_data[0], seed_data[1])

In [ ]:
for i in range(len(struct_data)):
  temp = []
  for j in seed_data[1][i]:
    temp.append(j['label'])
  temp = tuple(list(set(sorted(temp))))
  categorised_data[temp].append(struct_data[i])

for i in categorised_data:
  categorised_data[i] = list(set(categorised_data[i]))

In [ ]:
list_of_tags = {}
for i in range(len(seed_data[1])):
  for j in seed_data[1][i]:
    if j['label'] not in list_of_tags:
      list_of_tags[j['label']] = []
    list_of_tags[j['label']].append(seed_data[0][i][j['start']:j['end']])

In [ ]:
# def llm_response(prompt, sentence_examples):
#     openai = OpenAI(
#         api_key="bPdeUDQV6WUDGE6ifWQHult7iii7OWFj",
#         base_url="https://api.deepinfra.com/v1/openai",
#     )


#     chat_completion = openai.chat.completions.create(
#         model="mistralai/Mixtral-8x7B-Instruct-v0.1",
#         messages=[
#             {"role": "system", "content": f"You are given a set of with named entity recognition (NER) tags. Your task is to generate a set of 20 new sentences that contain exactly the same NER tags as given. Each new sentence should: 1. Use exactly the same NER tags in appropriate contexts without adding any new NER tags.  2. Reflect spoken language, as if someone is speaking to another person. 3. Follow the structure as given by {sentence_examples}. The format for the input will be such that you will get the NER tags in close brackets. Given this kind of input, you need to now generate 20 more sentence that contain exactly the same NER tags as given in the prompt. DO NOT ADD BULLET NUMBERS IN THE OUTPUT."},
#             {"role": "user", "content": prompt},
#         ],
#         temperature=1
#       )
#     return chat_completion.choices[0].message.content

In [ ]:
def llm_response(prompt, sentence_examples):
    openai = OpenAI(
        api_key="bPdeUDQV6WUDGE6ifWQHult7iii7OWFj",
        base_url="https://api.deepinfra.com/v1/openai",
    )


    chat_completion = openai.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=[
            {"role": "system", "content": f"You are given a set of with named entity recognition (NER) tags. Your task is to generate a set of 20 new sentences that contain exactly the same NER tags as given. Each new sentence should: 1. Use exactly the same NER tags in appropriate contexts without adding any new NER tags.  2. Reflect spoken language, as if someone is speaking to another person. 3. Be as creative and diverse as you can with the structure of sentences you come up with. Talk about the NER tag in different contexts and themes.. The format for the input will be such that you will get the NER tags in close brackets. Given this kind of input, you need to now generate 20 more sentence that contain exactly the same NER tags as given in the prompt. DO NOT ADD BULLET NUMBERS IN THE OUTPUT."},
            {"role": "user", "content": prompt},
        ],
        temperature=1
      )
    return chat_completion.choices[0].message.content

In [ ]:
def data_to_prompt(data, tags):
  prompt = ""
  prompt = "NER Tags = ("
  for tag in tags:
    prompt += data[tag['start']:tag['end']] + " - " + tag['label']
    prompt += ", "
  prompt = prompt + ")"
  return prompt

In [ ]:
def get_struct_sentences(sentence, tags):
  struct_sentences = []
  temp = []
  for j in tags:
    temp.append(j['label'])
  temp = tuple(list(set(sorted(temp))))
  if len(categorised_data[temp]) > 10:
    indices = random.sample(range(0, len(categorised_data[temp])), 10)
    for i in indices:
      struct_sentences.append(categorised_data[temp][i])
  else:
    struct_sentences = categorised_data[temp]
    for tag in temp:
      cat_data = categorised_data[tuple(list(set(sorted([tag]))))]
      indices = random.sample(range(0, len(cat_data)), 3)
      for i in indices:
        struct_sentences.append(cat_data[i])
  return struct_sentences

In [ ]:
augmented_dataset = []
for i in tqdm(range(len(seed_data[0]))):
  prompt = data_to_prompt(seed_data[0][i], seed_data[1][i])
  response = llm_response(prompt, "")
  augmented_dataset.append(response)

100%|██████████| 574/574 [1:20:03<00:00,  8.37s/it]


In [ ]:
with open('augmented_diverse_structure.pkl', 'wb') as f:
  pickle.dump(augmented_dataset, f)

In [ ]:
from google.colab import files
files.download('/content/augmented_diverse_structure.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>